# Introduction

This notebook shows how to model, create, and use a Snowflake Semantic View on top of TPC-DS sample data. You will define the model in both SQL and YAML, convert between the two, run example queries, and integrate the model with Snowflake AI features.

## What you will do
- Prepare simple views over shared TPC-DS tables.
- Define a semantic model in YAML and SQL; validate and create a Semantic View.
- Convert between YAML and Semantic Views (read/write).
- Query the Semantic View using dimensions, metrics, and filters.
- Integrate with Cortex Analyst and enhance natural language understanding with Cortex Search.
- [Optional] Connect Semantic View from 3rd party tools such as Omni
- [Optional] Share Semantic View Objects with other SF account


## Prerequisites
- A Snowflake account and warehouse.
- Privileges to create databases, schemas, stages, and semantic views.

## Outcome
By the end, you will have a working Semantic View (`TPCDS_SEMANTIC_VIEW_SM`) and a repeatable workflow to manage it via YAML or SQL.

### Part 1: Prepare Data for a Meaningful Model

In this section, you will use publicly available Snowflake sample data (via a share) to create simple views that the Semantic View will reference later.

In [ ]:
USE SCHEMA TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL;

In [ ]:
-- Create or replace views for the tables from SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL
CREATE OR REPLACE VIEW CUSTOMER AS
SELECT * FROM SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.CUSTOMER;

CREATE OR REPLACE VIEW CUSTOMER_DEMOGRAPHICS AS
SELECT * FROM SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.CUSTOMER_DEMOGRAPHICS;

CREATE OR REPLACE VIEW DATE_DIM AS
SELECT * FROM SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.DATE_DIM;

CREATE OR REPLACE VIEW ITEM AS
SELECT * FROM SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.ITEM;

CREATE OR REPLACE VIEW STORE AS
SELECT * FROM SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.STORE;

CREATE OR REPLACE VIEW STORE_SALES AS
SELECT * FROM SNOWFLAKE_SAMPLE_DATA.TPCDS_SF10TCL.STORE_SALES;

-- Query the Data
SELECT * FROM CUSTOMER LIMIT 20;

### Part 2: Define the Semantic Model and Create a Semantic View
This section reviews a YAML-based semantic model and uses it to create a Semantic View.

### 2-1. Review the YAML file
A SQL code checks whether a Semantic Model is stored in the "YAML" stage.

In [ ]:
-- List files in the YAML stage
ls @YAML;

### 2-2. How to review YAML file via Snowsight UI

This step involves reviewing semantic model definitions using Snowsight.

1. Sign in to Snowsight.
2. Navigate to **AI & ML** » **Cortex Analyst**.
3. Click Database, Schema, and Stage from the top columns. 
4. Choose **TKO_APJ_SEMANTIC_VIEW** database with **TPCDS_SF10TCL** schema and **YAML** stage.
5. Click your semantic model named `TPCDS_SEMANTIC_VIEW.yaml`
6. Click **“Edit YAML”** at the top and review definitions of a semantic model. 

### 2-3. Create Semantic View via Snowsight UI

This step involves converting a YAML-based semantic mode with a Semantic View object using Snowsight.

1. Sign in to Snowsight.
2. Navigate to **AI & ML** » **Cortex Analyst**.
3. Click **Database, Schema, and Stage** from the top columns. 
4. Choose **TKO_APJ_SEMANTIC_VIEW** database with **TPCDS_SF10TCL** schema and **YAML** stage.
5. Click the ellipsis (**...**) on the right and select "Convert to Semantic View".

### (Optional)2-3. Create a Semantic View via SQL
This SQL example mirrors the YAML-defined model for comparison.

In [ ]:
USE SCHEMA TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL;

CREATE OR REPLACE SEMANTIC VIEW TPCDS_SEMANTIC_VIEW_SM
	tables (
		CUSTOMER primary key (C_CUSTOMER_SK),
		DATE as DATE_DIM primary key (D_DATE_SK),
		DEMO as CUSTOMER_DEMOGRAPHICS primary key (CD_DEMO_SK),
		ITEM primary key (I_ITEM_SK),
		STORE primary key (S_STORE_SK),
		STORESALES as STORE_SALES
        primary key (SS_SOLD_DATE_SK,SS_CDEMO_SK,SS_ITEM_SK,SS_STORE_SK,SS_CUSTOMER_SK)
	)
	relationships (
		SALESTOCUSTOMER as STORESALES(SS_CUSTOMER_SK) references CUSTOMER(C_CUSTOMER_SK),
		SALESTODATE as STORESALES(SS_SOLD_DATE_SK) references DATE(D_DATE_SK),
		SALESTODEMO as STORESALES(SS_CDEMO_SK) references DEMO(CD_DEMO_SK),
		SALESTOITEM as STORESALES(SS_ITEM_SK) references ITEM(I_ITEM_SK),
		SALETOSTORE as STORESALES(SS_STORE_SK) references STORE(S_STORE_SK)
	)
	facts (
		ITEM.COST as i_wholesale_cost,
		ITEM.PRICE as i_current_price,
		STORE.TAX_RATE as S_TAX_PRECENTAGE,
        STORESALES.SALES_QUANTITY as SS_QUANTITY
	)
	dimensions (
		CUSTOMER.BIRTHYEAR as C_BIRTH_YEAR,
		CUSTOMER.COUNTRY as C_BIRTH_COUNTRY,
		CUSTOMER.C_CUSTOMER_SK as c_customer_sk,
		DATE.DATE as D_DATE,
		DATE.D_DATE_SK as d_date_sk,
		DATE.MONTH as D_MOY,
		DATE.WEEK as D_WEEK_SEQ,
		DATE.YEAR as D_YEAR,
		DEMO.CD_DEMO_SK as cd_demo_sk,
		DEMO.CREDIT_RATING as CD_CREDIT_RATING,
		DEMO.MARITAL_STATUS as CD_MARITAL_STATUS,
		ITEM.BRAND as I_BRAND,
		ITEM.CATEGORY as I_CATEGORY,
		ITEM.CLASS as I_CLASS,
		ITEM.I_ITEM_SK as i_item_sk,
		STORE.MARKET as S_MARKET_ID,
		STORE.SQUAREFOOTAGE as S_FLOOR_SPACE,
		STORE.STATE as S_STATE,
		STORE.STORECOUNTRY as S_COUNTRY,
		STORE.S_STORE_SK as s_store_sk,
		STORESALES.SS_CDEMO_SK as ss_cdemo_sk,
		STORESALES.SS_CUSTOMER_SK as ss_customer_sk,
		STORESALES.SS_ITEM_SK as ss_item_sk,
		STORESALES.SS_SOLD_DATE_SK as ss_sold_date_sk,
		STORESALES.SS_STORE_SK as ss_store_sk
	)
	metrics (
		STORESALES.TOTALCOST as SUM(item.cost),
		STORESALES.TOTALSALESPRICE as SUM(SS_SALES_PRICE),
		STORESALES.TOTALSALESQUANTITY as SUM(SS_QUANTITY)
            WITH SYNONYMS = ( 'total sales quantity', 'total sales amount')
	)
    
;


### 2-4. Inspect the Semantic View (SQL)
Let’s examine the following SQL function to check definitions of Semantic View.


In [ ]:
select get_ddl('SEMANTIC_VIEW', 'TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL.TPCDS_SEMANTIC_VIEW_SM',true);

### Tips – Snowsight vs SQL –

It is recommended that the initial creation of a Semantic model be performed using **Snowsight UI**, as it automatically generates descriptions for tables and dimensions. Additionally, while currently in public preview, it suggests additions to a Verified Query List. The Snowsight UI facilitates the definition of a Semantic model; however, updates to a **Semantic View are executed via SQL code** from CI/CD pipelines. 

## [Optional] Part 3: Convert Between YAML and Semantic Views
***This section is optional. If you have already created the semantic view in the previous section, you can skip Part 3.***

In this section, you will learn how to convert a semantic model between a YAML file and Semantic View object. Snowflake supports both SQL functions and a Snowsight UI flow for conversions.

### 3-1. YAML to Semantic View
You can convert a YAML file into a Semantic View in two ways. Please choose either one.

Option 1: Using Snowsight (UI)
1. Sign in to **Snowsight**.
2. Navigate to **AI & ML** » **Cortex Analyst**.
3. Select **Semantic models** tabs on the top
3. Click **Database, Schema, and Stage** from the top columns. 
4. Choose **TKO_APJ_SEMANTIC_VIEW** database with **TPCDS_SF10TCL** schema and **YAML** stage.
5. Click the ellipsis (...) on the right and select "**Convert to Semantic View**".


### Option 2: Using SQL Functions

The function `SYSTEM$CREATE_SEMANTIC_VIEW_FROM_YAML` can create a semantic view object from a YAML-format semantic model. However, it cannot read a YAML file directly from a stage. Let us share with you that a **custom Stored Procedure** below is used to read the YAML from the stage and then call the function with the YAML text.


In [ ]:
CREATE OR REPLACE PROCEDURE CREATE_SEMANTIC_VIEW_FROM_STAGE_YAML(
    TARGET_SCHEMA STRING,          -- e.g. 'MY_DB.MY_SCHEMA' (must be fully-qualified)
    STAGE_FILE_URL STRING,         -- e.g. '@MY_DB.MY_SCHEMA.MY_STAGE/path/model.yaml'
    VERIFY_ONLY BOOLEAN DEFAULT FALSE
)
RETURNS STRING
LANGUAGE JAVASCRIPT
EXECUTE AS CALLER
AS
$$
function exec(sql, binds) {
  var s = snowflake.createStatement({ sqlText: sql, binds: binds || [] });
  return s.execute();
}

// Split '<DB>.<SCHEMA>'
function splitSchema(fqSchema) {
  var parts = (fqSchema || '').split('.');
  if (parts.length !== 2) {
    throw new Error("TARGET_SCHEMA must be fully qualified as 'DB.SCHEMA': " + fqSchema);
  }
  return { db: parts[0], schema: parts[1] };
}

// Extract the semantic view name from YAML (first top-level 'name:' line).
function extractViewNameFromYaml(yamlText) {
  if (!yamlText) throw new Error("YAML text is empty.");
  var re = /^[ \t]*name[ \t]*:[ \t]*(?:"([^"]+)"|'([^']+)'|([^\r\n#]+))/mi;
  var m = yamlText.match(re);
  if (!m) {
    throw new Error("Could not find a top-level 'name:' in the YAML specification.");
  }
  var raw = (m[1] || m[2] || m[3] || "").trim();
  raw = raw.replace(/[ \t]+#[^\r\n]*$/, "").trim();
  return raw;
}

var ctx = splitSchema(TARGET_SCHEMA);

// 1) Create a temporary file format to read YAML as a single column ($1).
var ff_name = 'FF_YAML_' + Date.now();
var ff_fqn  = ctx.db + '.' + ctx.schema + '.' + ff_name;

exec(
  'CREATE TEMPORARY FILE FORMAT ' + ff_fqn +
  ' TYPE = CSV FIELD_DELIMITER = NONE RECORD_DELIMITER = NONE'
);

// 2) Read YAML from stage using the named file format.
var yamlText;
try {
  var rs = exec(
    'SELECT $1 AS YAML_TEXT FROM ' + STAGE_FILE_URL + ' (FILE_FORMAT => ?) LIMIT 1',
    [ff_fqn]
  );
  if (!rs.next()) {
    throw new Error('Failed to read YAML from stage: ' + STAGE_FILE_URL);
  }
  yamlText = String(rs.getColumnValue(1));
} finally {
  // Best-effort cleanup (TEMP also drops at session end).
  try { exec('DROP FILE FORMAT IF EXISTS ' + ff_fqn); } catch (e) {}
}

// 3) Determine the target semantic view FQN from YAML.
var viewNameFromYaml = extractViewNameFromYaml(yamlText);
var semanticViewFqn = ctx.db + '.' + ctx.schema + '.' + viewNameFromYaml;

// 4) Create or verify the semantic view from YAML.
var callRs = exec(
  'CALL SYSTEM$CREATE_SEMANTIC_VIEW_FROM_YAML(?, ?, ?)',
  [ (ctx.db + '.' + ctx.schema), yamlText, VERIFY_ONLY ]
);
callRs.next(); // status (not used further)

if (VERIFY_ONLY) {
  return 'VERIFIED: ' + semanticViewFqn;
} else {
  return 'CREATED: ' + semanticViewFqn;
}
$$;

In [ ]:
-- Execuiton
CALL CREATE_SEMANTIC_VIEW_FROM_STAGE_YAML(
  'TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL',
  '@TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL.YAML/TPCDS_SEMANTIC_VIEW.yaml',
  FALSE  -- set to TRUE to validate only
);

In [ ]:
-- Inspect the created Semantic View:
select get_ddl(
'SEMANTIC_VIEW',          
'TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL.TPCDS_SEMANTIC_VIEW_SM', 
true)

### 3-2. Semantic View to YAML

`SYSTEM$READ_YAML_FROM_SEMANTIC_VIEW` Function allows you to translate a YAML -based Semantic model from a semantic view object.

In [ ]:
-- Read the YAML representation of the Semantic View
SELECT SYSTEM$READ_YAML_FROM_SEMANTIC_VIEW('TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL.TPCDS_SEMANTIC_VIEW_SM') AS yaml_format

You can store a YAML file into a Snowflake stage by utilizing `SYSTEM$READ_YAML_FROM_SEMANTIC_VIEW` in conjunction with the **Copy File** command. Note that the parameters FIELD_DELIMITER, RECORD_DELIMITER, and FIELD_OPTIONALLY_ENCLOSED_BY within the FILE_FORMAT are configured as None during the creation of a YAML format.  

In [ ]:
-- Save the YAML to an internal stage file
COPY INTO @TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL.YAML/TPCDS_SEMANTIC_VIEW_2.yaml
FROM (
  SELECT SYSTEM$READ_YAML_FROM_SEMANTIC_VIEW('TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL.TPCDS_SEMANTIC_VIEW_SM')
)
FILE_FORMAT = (
  TYPE = CSV
  FIELD_DELIMITER = NONE 
  RECORD_DELIMITER = NONE
  FIELD_OPTIONALLY_ENCLOSED_BY = NONE
  COMPRESSION = NONE
)
SINGLE = TRUE
OVERWRITE = TRUE;

In [ ]:
-- Verify the file was saved to the stage
ls @YAML

## Part 4: Querying a Semantic View
Practice querying the Semantic View using dimensions, metrics, and filters expressed in business terms.
Example Query (Semantic View Syntax)


In [ ]:
-- Example query using the Semantic View syntax
SELECT * FROM SEMANTIC_VIEW
( 
 TPCDS_SEMANTIC_VIEW_SM
    DIMENSIONS 
            Item.Brand,
            Item.Category,            
            Date.Year,
            Date.Month,
            Store.State
    METRICS 
        StoreSales.TotalSalesQuantity
    WHERE
        Date.Year = '2002' AND Date.Month = '12' AND Store.State ='TX' AND Item.Category = 'Books'
) 
ORDER BY TotalSalesQuantity DESC LIMIT 10;

Let's break down each part:
- `SELECT * FROM SEMANTIC_VIEW (view_name ...)`: This initiates your query, indicating that you want to retrieve data from a semantic view
- `METRICS metric1, metric2, ...`: Within the parentheses, specify the metrics (calculated values or totals) you wish to retrieve
- `DIMENSIONS dimension1, dimension2, ...`: Next, specify the dimensions (categories or attributes) you want to group or filter your data by

You can enhance your queries with standard SQL clauses:

- Filtering: Use `WHERE` clauses to filter your results
- Sorting: Use `ORDER BY` to sort your results
- Limiting: Use `LIMIT` to restrict the number of results

## Part 5: Query Semantic Views with Cortex Analyst
This section focuses on enhancing Semantic View accuracy. You will learn to achieve this by leveraging natural language questions.

### 5-1. Access to Cortex Analyst

This step involves setting up Cortex Agents with Semantic View via Snowsight.  

1. Sign in to **Snowsight**.
2. Navigate to **AI & ML** » **Cortex Analyst**.
3. Select the “**Semantic views**” tab on the top.
4. Choose `TPCDS_SEMANTIC_VIEW_SM` in the list.
5. Select the “**Playground**” tab on the right.


### 5-2. Ask questions
Prior to making any changes to the semantic view, you will assess its current accuracy by asking a series of questions below. 

#### 1. Ask questions in Cortex Analyst UI
Copy and Paste a question from the following list into your playground, and run it

1. **Please display 10 sample records from the ITEM table where BRAND is ‘edu packunivamalg’**
2. **Show me metrics_x for each store in the US in 2003**
3. **Display all records from the STORESALES table**

#### 2. Check Answers:  
You may receive errors in each question.

**Option2: Snowsight UI**
1. Sign in to Snowsight.
2. Navigate to AI & ML → Agents.
3. Click "Create Agent" and follow the guided setup.
4. Associate your Semantic View or upload a YAML model when prompted.

### 5-3. Update Semantic View 

If a natural language query fails due to an exact match not being found, you can integrate **Cortex Search** for columns with many unique values to handle typos and partial matches.

#### Steps to Integrate Cortex Search

In [ ]:
-- 1. Create a source table for the search service. 
CREATE OR REPLACE TABLE ITEM_BRAND AS 
    SELECT DISTINCT I_BRAND FROM ITEM;

-- 2. Create the Cortex Search Service:
CREATE OR REPLACE CORTEX SEARCH SERVICE ITEM_BRAND_SEARCH_SERVICE
  ON I_BRAND
  WAREHOUSE = TKP_APJ_SEMANTIC_VIEW_WH
  TARGET_LAG = '1 day'
  EMBEDDING_MODEL = 'snowflake-arctic-embed-l-v2.0'
  AS (
    SELECT
        I_BRAND,
    FROM ITEM_BRAND
);

#### Recreate the Semantic View to include the Cortex Search Service:

Here is a SQL code to update the existing Semantic View.

In [ ]:
-- Add synonyms to improve natural language understanding in Cortex Agents

USE SCHEMA TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL;

CREATE OR REPLACE SEMANTIC VIEW TKO_APJ_SEMANTIC_VIEW.TPCDS_SF10TCL.TPCDS_SEMANTIC_VIEW_SM
	tables (
		CUSTOMER primary key (C_CUSTOMER_SK),
		DATE as DATE_DIM primary key (D_DATE_SK),
		DEMO as CUSTOMER_DEMOGRAPHICS primary key (CD_DEMO_SK),
		ITEM primary key (I_ITEM_SK),
		STORE primary key (S_STORE_SK),
		STORESALES as STORE_SALES primary key (SS_SOLD_DATE_SK,SS_CDEMO_SK,SS_ITEM_SK,SS_STORE_SK,SS_CUSTOMER_SK)
	)
	relationships (
		SALESTOCUSTOMER as STORESALES(SS_CUSTOMER_SK) references CUSTOMER(C_CUSTOMER_SK),
		SALESTODATE as STORESALES(SS_SOLD_DATE_SK) references DATE(D_DATE_SK),
		SALESTODEMO as STORESALES(SS_CDEMO_SK) references DEMO(CD_DEMO_SK),
		SALESTOITEM as STORESALES(SS_ITEM_SK) references ITEM(I_ITEM_SK),
		SALETOSTORE as STORESALES(SS_STORE_SK) references STORE(S_STORE_SK)
	)
	facts (
		ITEM.COST as i_wholesale_cost,
		ITEM.PRICE as i_current_price,
		STORE.TAX_RATE as S_TAX_PRECENTAGE,
		STORESALES.SALES_QUANTITY as SS_QUANTITY
	)
	dimensions (
		CUSTOMER.BIRTHYEAR as C_BIRTH_YEAR,
		CUSTOMER.COUNTRY as C_BIRTH_COUNTRY,
		CUSTOMER.C_CUSTOMER_SK as c_customer_sk,
		DATE.DATE as D_DATE,
		DATE.D_DATE_SK as d_date_sk,
		DATE.MONTH as D_MOY,
		DATE.WEEK as D_WEEK_SEQ,
		DATE.YEAR as D_YEAR,
		DEMO.CD_DEMO_SK as cd_demo_sk,
		DEMO.CREDIT_RATING as CD_CREDIT_RATING,
		DEMO.MARITAL_STATUS as CD_MARITAL_STATUS,
        -- Add Cortex Search 
		ITEM.BRAND as I_BRAND with cortex search service ITEM_BRAND_SEARCH_SERVICE,
		
        ITEM.CATEGORY as I_CATEGORY,
		ITEM.CLASS as I_CLASS,
		ITEM.I_ITEM_SK as i_item_sk,
		STORE.MARKET as S_MARKET_ID,
		STORE.SQUAREFOOTAGE as S_FLOOR_SPACE,
		STORE.STATE as S_STATE,
		STORE.STORECOUNTRY as S_COUNTRY,
		STORE.S_STORE_SK as s_store_sk,
		STORESALES.SS_CDEMO_SK as ss_cdemo_sk,
		STORESALES.SS_CUSTOMER_SK as ss_customer_sk,
		STORESALES.SS_ITEM_SK as ss_item_sk,
		STORESALES.SS_SOLD_DATE_SK as ss_sold_date_sk,
		STORESALES.SS_STORE_SK as ss_store_sk
	)
	metrics (
		STORESALES.TOTALCOST as SUM(item.cost),
		STORESALES.TOTALSALESPRICE as SUM(SS_SALES_PRICE),
		STORESALES.TOTALSALESQUANTITY as SUM(SS_QUANTITY) with synonyms=('metrics_x','total sales amount','total sales quantity') -- Add metrics_x
	)
	with extension (CA='{"module_custom_instructions":{"sql_generation":"Ensure output is limited to 10000 records"}}') -- Add custom instructions
;

### 5-4. Access to Cortex Analyst to ask same questions again
you will assess its improved accuracy by asking the same series of questions as 5-2. parts. Let’s move to Cortex Analyst UI. 

This step involves setting up Cortex Agents with Semantic View via Snowsight.  

1. Sign in to **Snowsight**.
2. Navigate to **AI & ML** » **Cortex Analyst**.
3. Select the “**Semantic views**” tab on the top.
4. Choose `TPCDS_SEMANTIC_VIEW_SM` in the list.
5. Select the “**Playground**” tab on the right.
6. Ask the same questions as part 5-2 and Check Answer 


[**Tips to improve performance for Semantic Model**]

Start with simple model tweaks (synonyms, samples) before adding Cortex Search or verified queries.

- ***Synonyms & descriptions***: Align column names and synonyms with user language; include key info like time zones.
- ***Multilingual synonyms***: Add synonyms in user languages (e.g., Japanese, Korean) for better query understanding.
- ***Sample values***: Provide examples to help the LLM link user terms (e.g., “Snowflake” vs “Snowflake Inc.”) to actual data.
- ***Cortex Search integration***: Use Search for columns with many unique values to handle typos and partial matches.
- ***Verified query repository***: Store trusted, complex SQL logic for reuse and onboarding suggestions.
- ***Custom instructions***: Use custom_instructions for clear behavioral rules; iterate gradually and test generated SQL.
- ***Semi-structured data***: Flatten when possible; otherwise, use fuzzy matching or VQR to extract nested elements.